<a href="https://colab.research.google.com/github/smraytech/test/blob/main/Telco_customer_churn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📌 Project Title

Customer Churn Prediction and Retention Insights for a Subscription Service

📌 Problem Statement

The company is losing customers every month but does not understand why they are leaving or which customers are at high risk of churn. This leads to revenue loss and high acquisition costs.

https://www.kaggle.com/datasets/blastchar/telco-customer-churn

Workflow for Churn Prediction (Imbalance-Aware) STEP 0 — Imports

In [2]:
# -----------------------------------------
# STEP 1 — Import libraries and load dataset
# -----------------------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import pickle

In [14]:
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [15]:
print("Loaded:", df.shape)

Loaded: (7043, 21)


✅ STEP B — Clean TotalCharges

In [16]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", None), errors="coerce")
df["TotalCharges"] = df["TotalCharges"].fillna(0)

✅ STEP C — Strip whitespace safely

In [17]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

✅ STEP D — Create target variable (NO NaN)

In [18]:
df["ChurnFlag"] = df["Churn"].map({"Yes": 1, "No": 0})
print("Missing in ChurnFlag:", df["ChurnFlag"].isna().sum())

Missing in ChurnFlag: 0


✅ STEP E — Prepare X and y

In [19]:
drop_cols = ["customerID", "Churn"]
X = df.drop(columns=drop_cols)
y = df["ChurnFlag"]

STEP 4

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)
print("Train churn rate:", y_train.mean())
print("Test churn rate:", y_test.mean())

Train size: (5634, 20)
Test size: (1409, 20)
Train churn rate: 0.2653532126375577
Test churn rate: 0.2654364797728886


✅ STEP 5 — Build a Preprocessing Pipeline

This pipeline will:

✔ Handle missing numeric values
✔ Scale numeric features
✔ Impute missing categorical values
✔ One-hot encode categoricals
✔ Avoid the OneHotEncoder sparse error
✔ Work perfectly with SMOTE later

✅ STEP 5 — Identify Numeric & Categorical Columns

In [22]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)

Numeric columns: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'ChurnFlag']
Categorical columns: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


✅ STEP 5B — Build the Numeric Transformer

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [24]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [25]:
numeric_transformer

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

✅ STEP 5C — Build the Categorical Transformer

⚠ Important:
Use sparse_output=False (NOT sparse=False

In [26]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [27]:
categorical_transformer

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehot',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

✅ STEP 5D — Combine Using ColumnTransformer

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ],
    remainder='drop'
)


In [30]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['SeniorCitizen', 'tenure', 'MonthlyCharges',
                                  'TotalCharges', 'ChurnFlag']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['gender', 'Partner', 'Dependents',
                                  'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'Contract',
                                  'PaperlessBilling', 'PaymentMethod'])])

In [31]:
print("Preprocessor created successfully.")

Preprocessor created successfully.


✅ STEP 6 — Train Logistic Regression (with our clean pipeline)

This will:

✔ Use your preprocessing pipeline
✔ Fit the model
✔ Produce clean predictions
✔ Produce evaluation metrics (Accuracy, Precision, Recall, F1)

✅ STEP 6A — Build Logistic Regression Pipeline

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [33]:
log_reg_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=500))
])

In [34]:
log_reg_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('classifier', LogisticRegression(max_iter=500))])

✅ STEP 6B — Train the Model

In [35]:
log_reg_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('classifier', LogisticRegression(max_iter=500))])

✅ STEP 6C — Predict on Test Data

In [36]:
log_pred = log_reg_model.predict(X_test)

In [37]:
log_pred

array([0, 0, 0, ..., 0, 0, 0])

✅ STEP 6D — Evaluate Metrics

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [39]:
log_accuracy  = accuracy_score(y_test, log_pred)
log_precision = precision_score(y_test, log_pred)
log_recall    = recall_score(y_test, log_pred)
log_f1        = f1_score(y_test, log_pred)

print("=== Logistic Regression Performance ===")
print("Accuracy :", log_accuracy)
print("Precision:", log_precision)
print("Recall   :", log_recall)
print("F1 Score :", log_f1)

=== Logistic Regression Performance ===
Accuracy : 1.0
Precision: 1.0
Recall   : 1.0
F1 Score : 1.0


✅ STEP 7 — Train Random Forest Model (with the same preprocessing pipeline)

This model usually performs better than Logistic Regression for churn prediction.

✅ STEP 7A — Build Random Forest Pipeline

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ))
])

In [42]:
rf_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...own='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('classifier',
                 RandomForestClassifier(n_estimators=300, n_jobs=-1,
                                        random_state=42))])

✅ STEP 7B — Train the Model

In [43]:
rf_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...own='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('classifier',
                 RandomForestClassifier(n_estimators=300, n_jobs=-1,
                                        random_state=42))])

✅ STEP 7C — Predict on Test Data

In [44]:
rf_pred = rf_model.predict(X_test)

In [45]:
rf_pred

array([0, 0, 0, ..., 0, 0, 0])

✅ STEP 7D — Evaluate Metrics

In [46]:
rf_accuracy  = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall    = recall_score(y_test, rf_pred)
rf_f1        = f1_score(y_test, rf_pred)

print("=== Random Forest Performance ===")
print("Accuracy :", rf_accuracy)
print("Precision:", rf_precision)
print("Recall   :", rf_recall)
print("F1 Score :", rf_f1)

=== Random Forest Performance ===
Accuracy : 1.0
Precision: 1.0
Recall   : 1.0
F1 Score : 1.0


✅ STEP 8 — Train XGBoost Model (with preprocessing pipeline)

XGBoost normally gives the strongest churn prediction performance, especially with imbalanced data.

⚠️ IMPORTANT

Before running XGBoost, install the package if it's missing:

In [47]:
!pip install xgboost

✅ STEP 8A — Import XGBoost

In [48]:
import xgboost as xgb

In [49]:
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        eval_metric='logloss',
        random_state=42
    ))
])

In [50]:
xgb_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=None,
                               num_parallel_tree=None, ...))])

✅ STEP 8B — Build XGBoost Pipeline

In [52]:
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        eval_metric='logloss',
        random_state=42
    ))
])

In [53]:
xgb_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=None,
                               num_parallel_tree=None, ...))])

✅ STEP 8C — Train the Model

In [55]:
xgb_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'ChurnFlag']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=None,
                               num_parallel_tree=None, ...))])

✅ STEP 8D — Predict

In [56]:
xgb_pred = xgb_model.predict(X_test)

In [57]:
xgb_pred

array([0, 0, 0, ..., 0, 0, 0])

✅ STEP 8E — Evaluate

In [58]:
xgb_accuracy  = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred)
xgb_recall    = recall_score(y_test, xgb_pred)
xgb_f1        = f1_score(y_test, xgb_pred)

print("=== XGBoost Performance ===")
print("Accuracy :", xgb_accuracy)
print("Precision:", xgb_precision)
print("Recall   :", xgb_recall)
print("F1 Score :", xgb_f1)

=== XGBoost Performance ===
Accuracy : 1.0
Precision: 1.0
Recall   : 1.0
F1 Score : 1.0


✅ STEP 9 — Compare All Models (Logistic, Random Forest, XGBoost)

You now have predictions from:

log_reg_model

rf_model

xgb_model

Next, we build a simple comparison table to see which model performs best.

✅ STEP 9A — Generate metrics for each model

Make sure you already have these predictions:

In [59]:
log_pred = log_reg_model.predict(X_test)
rf_pred  = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

Now compute and store their performance:

In [60]:
models = ["Logistic Regression", "Random Forest", "XGBoost"]
accuracies  = []
precisions  = []
recalls     = []
f1_scores   = []

for pred in [log_pred, rf_pred, xgb_pred]:
    accuracies.append(accuracy_score(y_test, pred))
    precisions.append(precision_score(y_test, pred))
    recalls.append(recall_score(y_test, pred))
    f1_scores.append(f1_score(y_test, pred))

✅ STEP 9B — Create a comparison DataFrame

In [61]:
comparison_df = pd.DataFrame({
    "Model": models,
    "Accuracy": accuracies,
    "Precision": precisions,
    "Recall": recalls,
    "F1 Score": f1_scores
})

comparison_df

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,1.0,1.0,1.0,1.0
1,Random Forest,1.0,1.0,1.0,1.0
2,XGBoost,1.0,1.0,1.0,1.0


✅ STEP 9C — Display the best model

In [62]:
best_model_idx = comparison_df["F1 Score"].idxmax()
best_model_name = comparison_df.iloc[best_model_idx]["Model"]

print("🔥 BEST MODEL:", best_model_name)

🔥 BEST MODEL: Logistic Regression


best_model_idx = comparison_df["F1 Score"].idxmax()
best_model_name = comparison_df.iloc[best_model_idx]["Model"]

print("🔥 BEST MODEL:", best_model_name)

✅ STEP 10A — Map model names to model objects

Add this block after Step 9:

In [63]:
model_objects = {
    "Logistic Regression": log_reg_model,
    "Random Forest": rf_model,
    "XGBoost": xgb_model
}

✅ STEP 10B — Identify the best-performing model

In [64]:
best_model_name = comparison_df.iloc[comparison_df["F1 Score"].idxmax()]["Model"]
best_model = model_objects[best_model_name]

print("🔥 BEST MODEL SELECTED:", best_model_name)

🔥 BEST MODEL SELECTED: Logistic Regression


✅ STEP 10C — Save the Preprocessor + Best Model as One Pipeline

We wrap everything into a single pipeline object so that deployment is very easy.

In [65]:
import pickle

In [68]:
# best_model is already a pipeline: preprocessor + classifier
# Save it directly
with open("churn_best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("🎉 SUCCESS: Model saved as churn_best_model.pkl")

🎉 SUCCESS: Model saved as churn_best_model.pkl


In [70]:
type(best_model)

sklearn.pipeline.Pipeline

In [80]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


In [81]:
numeric_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport']

In [83]:
from sklearn import __version__ as sklearn_version

import packaging

In [84]:
if packaging.version.parse(sklearn_version) >= packaging.version.parse("1.2"):
    # Newer versions use sparse_output
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
else:
    # Older versions use sparse
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Now use it in the categorical pipeline
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('onehot', ohe)
])

This ensures your pipeline works regardless of scikit-learn version.

After this, the ColumnTransformer and full pipeline can be created exactly as before:

In [85]:
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

churn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_estimators=200, random_state=42))
])